In [1]:
import torch
import sys
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split

print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

import numpy
import numpy as np

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

import pandas
import pandas as pd

import logging
handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)
%matplotlib inline

# !pip install psutil
import psutil
import os
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)

cpuStats()

__Python VERSION: 3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
__pyTorch VERSION: 1.2.0
__CUDA VERSION
/bin/sh: 1: nvcc: not found
__CUDNN VERSION: 7301
__Number CUDA Devices: 2
__Devices
Active CUDA Device: GPU 0
Available devices  2
Current cuda device  0
3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
12.7
svmem(total=67477729280, available=44222099456, percent=34.5, used=22014279680, free=28895248384, active=24637640704, inactive=7641436160, buffers=82919424, cached=16485281792, shared=485761024, slab=5041659904)
memory GB: 0.24361801147460938


In [2]:
use_cuda = torch.cuda.is_available()
# use_cuda = False

FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

In [3]:
os.chdir('data')

In [4]:
!pwd

/projects/audio-pipelines/data


In [5]:
# hyps = pd.read_csv("sets/full_set/hyps", names=['audio_id'])
# hyps[['audio_id','hyps']] = hyps["audio_id"].str.split(" ", 1, expand=True)

# targets = pd.read_csv("sets/full_set/targets", names=['audio_id'])
# targets[['audio_id','target']] = targets["audio_id"].str.split(" ", 1, expand=True)

# text = pd.read_csv("sets/full_set/text", names=['audio_id'])
# text[['audio_id','text']] = text["audio_id"].str.split(" ", 1, expand=True)

# speakers = pd.read_csv("sets/full_set/utt2spk", names=['audio_id'])
# speakers[['audio_id','speaker']] = speakers["audio_id"].str.split(" ", 1, expand=True)

# images = pd.read_csv("sets/full_set/wav.scp", names=['audio_id'])
# images[['audio_id','audio_path']] = images["audio_id"].str.split(" ", 1, expand=True)

# dataset = pd.merge(hyps, text, how="left")
# dataset = pd.merge(dataset, speakers, how="left")
# dataset = pd.merge(dataset, images, how="left")
# dataset = pd.merge(dataset, targets, how="left")

In [6]:
# dataset['image_path'] = dataset['audio_path'].apply(lambda x : x[:-4] + ".png")
# def remove_absolute(string, prefix='/home/raznem/projects/audio-pipelines/data/'):
#     if string.startswith(prefix):
#         string = string[len(prefix):]
#     return string
    
# dataset['image_path'] = dataset['image_path'].apply(remove_absolute)

In [7]:
# dataset.head()

In [8]:
# dataset.to_csv('data/dataset.csv')

In [9]:
## from allennlp.commands.elmo import ElmoEmbedder
from torchvision import transforms
import numpy as np

try:
    from PIL import Image
except ImportError:
    import Image
    
toImg = transforms.ToPILImage()
toTensor = transforms.ToTensor()
    
class WavImagesLoader(Dataset):
    def __init__(self, csv_path, transform=None):
        self.dataset = pd.read_csv(csv_path)
        self.transform = transform
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image = Image.open(image_path)
        image = image.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)
        
        return image, target
    
    def __len__(self):
        return len(self.dataset)
    
    @staticmethod        
    def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))    

#     @staticmethod    
#     def flaotTensorToImage(img, mean=0, std=1):
#         """convert a tensor to an image"""
#         img = np.transpose(img.numpy(), (1, 2, 0))
#         img = (img*std+ mean)*255
#         img = img.astype(np.uint8)    
#         return img    
    
    
class ElmoWavImagesLoader(WavImagesLoader):
    def __init__(self, csv_path, text_vecs_path, transform=None):
        super().__init__(csv_path, transform=None)
        self.text_vecs_path = text_vecs_path
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image = Image.open(image_path)
        image = image.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)
        
        text_vec = np.zeros((322, 1024))
        tmp = np.load(self.text_vecs_path + self.dataset["audio_id"][key] + ".npy")
        tmp = np.sum(tmp, axis=0)
        text_len = tmp.shape[0]
        text_vec[:tmp.shape[0],:tmp.shape[1]] = tmp
        
        return image, text_vec, text_len, target
    
    
class ElmoWavVecLoader(ElmoWavImagesLoader):
    def __init__(self, csv_path, text_vecs_path, transform=None):
        super().__init__(csv_path, text_vecs_path, transform=None)
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_vec = self.dataset['image_vec'][key]
        text = self.dataset['hyps'][key]
        text = text.split(' ')
        
        text_vec = np.zeros(322, 1024)
        tmp = np.load(text_vecs_path + self.dataset["audio_id"][key] + ".npy")
        text_len = tmp.shape[0]
        text_vec[:tmp.shape[0],:tmp.shape[1]] = tmp

        return image_vec, text_vec, text_len, target
    
class ElmoWavImagesLoader_v2(WavImagesLoader):
    def __init__(self, csv_path, text_vec_list, transform=None):
        super().__init__(csv_path, transform=None)
        self.text_vec_list = text_vec_list
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image = Image.open(image_path)
        image = image.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)

        text_vec = np.zeros((322, 1024))
        tmp = self.text_vec_list[key]
        text_len = tmp.shape[0]
        text_vec[:tmp.shape[0],:tmp.shape[1]] = tmp

        return image, text_vec, text_len, target

## Conv net to get vector from Image

In [10]:
dropout = torch.nn.Dropout(p=0.30)
class ConvRes(nn.Module):
    def __init__(self, insize, outsize):
        super(ConvRes, self).__init__()
        drate = .3
        self.math = nn.Sequential(
            nn.BatchNorm2d(insize),
            # nn.Dropout(drate),
            torch.nn.Conv2d(insize, outsize, kernel_size=2, padding=2),
            nn.PReLU(),
        )

    def forward(self, x):
        return self.math(x)


class ConvCNN(nn.Module):
    def __init__(self, insize, outsize, kernel_size=7, padding=2, pool=2, avg=True):
        super(ConvCNN, self).__init__()
        self.avg = avg
        self.math = torch.nn.Sequential(
            torch.nn.Conv2d(insize, outsize, kernel_size=kernel_size, padding=padding),
            torch.nn.BatchNorm2d(outsize),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(pool, pool),
        )
        self.avgpool = torch.nn.AvgPool2d(pool, pool)

    def forward(self, x):
        x = self.math(x)
        if self.avg is True:
            x = self.avgpool(x)
        return x


class Net(nn.Module):
    def __init__(self, output_dim):
        super(Net, self).__init__()

        self.avgpool = nn.AdaptiveAvgPool2d(1)

        self.cnn1 = ConvCNN(3, 32, kernel_size=7, pool=4, avg=False)
        self.cnn2 = ConvCNN(32, 32, kernel_size=5, pool=2, avg=True)
        self.cnn3 = ConvCNN(32, 32, kernel_size=5, pool=2, avg=True)

        self.res1 = ConvRes(32, 64)

        self.features = nn.Sequential(
            self.cnn1, dropout,
            self.cnn2,
            self.cnn3,
            self.res1,
        )

        self.classifier = torch.nn.Sequential(
            nn.Linear(5184, output_dim),
        )
        self.sig = nn.Sigmoid()
  
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
#         x = self.sig(x)
        return x

    
# if use_cuda:
#     lgr.info ("Using the GPU")
#     model = Net(output_dim).cuda() # On GPU
# else:
#     lgr.info ("Using the CPU")
#     model = Net(output_dim) # On CPU

# lgr.info('Model {}'.format(model))

Get maximum length of text

In [11]:
# max_text_length = dataset_dropna['hyps'].apply(lambda x: len(x.split(' '))).max()
# max_text_length

## RNN to encode Elmo sequence

In [12]:
class TextSpecModel(Net):
    def __init__(self, cnn_dim, rnn_dim, layers_rnn=1, text_emb_dim=1024):
        super(Net, self).__init__()
        self.layers_rnn = layers_rnn
        self.text_emb_dim = text_emb_dim
        self.rnn_dim = rnn_dim
        self.cnn_model = Net(cnn_dim)
        self.text_lstm = nn.LSTM(text_emb_dim, self.rnn_dim, num_layers=self.layers_rnn, batch_first=True)
        self.fc_last = nn.Linear(cnn_dim + rnn_dim, 1)
        self.sig = nn.Sigmoid()
        
    def forward(self, img, text, text_len):        
        cnn_out = self.cnn_model(img)
        seq_lengths = text_len
        pack = torch.nn.utils.rnn.pack_padded_sequence(text, seq_lengths, batch_first=True)
        _, (rnn_out, _) = self.text_lstm(text)
        rnn_out = rnn_out.reshape(-1, self.rnn_dim)
        x = torch.cat([cnn_out, rnn_out], dim=1)
        y_pred = self.fc_last(x)
        y_pred = torch.squeeze(y_pred)
        y_pred = self.sig(y_pred)
        return y_pred

## Training function

In [13]:
import gc
def textspec_trainer(loader_train, model, optimizer, exp_name='', save_every=None, print_every=None, epochs=1, use_gpu=True, 
                 dtype=torch.float32):
    if use_gpu and torch.cuda.is_available():
        device = torch.device('cuda:1')
    else:
        device = torch.device('cpu')
        
    model.train()
    model.to(device=device)

    for e in range(epochs):
        print('Epoch %d' %e)
        acc = 0
        files_no = 0
        for t, (img, text, text_len, y) in enumerate(loader_train):
            img = img.to(device=device, dtype=dtype)
            text = text.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=dtype)
            text_len, perm_idx = text_len.sort(0, descending=True)
            text = text[perm_idx]
            img = img[perm_idx]
            y = y[perm_idx]
            
            y_pred = model(img, text, text_len)
            
            loss = loss_fn(y_pred, y)
            
            acc += (torch.round(y_pred.cpu()) == y.cpu()).sum().type(torch.float32)
            files_no += len(text)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            if print_every is not None and t % print_every == 0:
                print('Iteration %d, loss = %.4f; epoch %d' % (t, loss.item(), e))
                print(f'Acc {acc / files_no}')
                
                
        if save_every is not None and e % save_every == 0:
            torch.save(model.state_dict(), f'models/{exp_name}_e%d_cnn_rnn.pt' % e)
            gc.collect()
    torch.save(model.state_dict(), f'models/{exp_name}_cnn_rnn.pt')

## Training

In [15]:
import pickle as pkl

BATCH_SIZE = 32
LEARNING_RATE = 1e-4
USE_GPU = True
TE_DTYPE = torch.float32
CNN_DIM = 128
RNN_DIM = 128
NUM_EPOCHS = 100

with open("text_vec_list.p", "rb") as f:
    text_vec_list = pkl.load(f)
loss_fn = torch.nn.BCELoss()
model = TextSpecModel(CNN_DIM, RNN_DIM)
image_data = ElmoWavImagesLoader_v2('data/dataset_dropna.csv', text_vec_list, '../text_vecs/')
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loader_train = DataLoader(image_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=12)
textspec_trainer(loader_train, model, optimizer, exp_name='textspec', save_every=10, print_every=200, epochs=NUM_EPOCHS,
      use_gpu=USE_GPU, dtype=TE_DTYPE)

Epoch 0
Iteration 0, loss = 0.6966; epoch 0
Acc 0.4375
Iteration 200, loss = 0.6780; epoch 0
Acc 0.6026119589805603
Iteration 400, loss = 0.6466; epoch 0
Acc 0.6083229184150696
Iteration 600, loss = 0.6266; epoch 0
Acc 0.6111688613891602
Iteration 800, loss = 0.6910; epoch 0
Acc 0.6138030290603638
Epoch 1
Iteration 0, loss = 0.6068; epoch 1
Acc 0.6875
Iteration 200, loss = 0.6322; epoch 1
Acc 0.638059675693512
Iteration 400, loss = 0.6456; epoch 1
Acc 0.6329488754272461
Iteration 600, loss = 0.6449; epoch 1
Acc 0.6298356652259827
Iteration 800, loss = 0.6499; epoch 1
Acc 0.6287453174591064
Epoch 2
Iteration 0, loss = 0.6312; epoch 2
Acc 0.65625
Iteration 200, loss = 0.6997; epoch 2
Acc 0.6295087337493896
Iteration 400, loss = 0.6504; epoch 2
Acc 0.634585440158844
Iteration 600, loss = 0.5898; epoch 2
Acc 0.6326954960823059
Iteration 800, loss = 0.5309; epoch 2
Acc 0.634402334690094
Epoch 3
Iteration 0, loss = 0.7266; epoch 3
Acc 0.59375
Iteration 200, loss = 0.6318; epoch 3
Acc 0.63821

OSError: [Errno 12] Cannot allocate memory